In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import time

class EmotionClassifier:
    def __init__(self, model_name="bhadresh-savani/distilbert-base-uncased-emotion"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model.eval()
        
        # Map ID to emotion label
        self.id2label = {
            0: "sadness",
            1: "joy", 
            2: "love", 
            3: "anger", 
            4: "fear", 
            5: "surprise"
        }
    
    def classify(self, text):
        # Tokenize and prepare input
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        
        # Perform inference
        with torch.inference_mode():
            outputs = self.model(**inputs)
            
        # Get predictions
        scores = torch.nn.functional.softmax(outputs.logits, dim=1)
        predictions = torch.argmax(scores, dim=1)
        
        # Map to emotion labels
        emotion = self.id2label[predictions.item()]
        confidence = scores[0][predictions.item()].item()
        
        return {
            "emotion": emotion,
            "confidence": confidence,
            "all_scores": {self.id2label[i]: score.item() for i, score in enumerate(scores[0])}
        }

# Test the classifier
classifier = EmotionClassifier()

# Example sentences
test_sentences = [
    "He have been working on this project for three year.",
    "I'm so happy about this new opportunity!",
    "I'm really disappointed with the service we received.",
    "That movie was so terrifying, I couldn't sleep."
]

# Test performance
start_time = time.time()
for sentence in test_sentences:
    result = classifier.classify(sentence)
    print(f"Sentence: '{sentence}'")
    print(f"Emotion: {result['emotion']} (confidence: {result['confidence']:.4f})")
    print(f"All emotions: {result['all_scores']}")
    print()

print(f"Total time for {len(test_sentences)} sentences: {time.time() - start_time:.4f} seconds")

Sentence: 'He have been working on this project for three year.'
Emotion: joy (confidence: 0.5305)
All emotions: {'sadness': 0.015558494254946709, 'joy': 0.5305135250091553, 'love': 0.0036071939393877983, 'anger': 0.2742093503475189, 'fear': 0.17232690751552582, 'surprise': 0.0037843899335712194}

Sentence: 'I'm so happy about this new opportunity!'
Emotion: joy (confidence: 0.9987)
All emotions: {'sadness': 0.0003589358530007303, 'joy': 0.9987414479255676, 'love': 0.00038312148535624146, 'anger': 0.00022281201381701976, 'fear': 0.0001380163594149053, 'surprise': 0.0001556968636577949}

Sentence: 'I'm really disappointed with the service we received.'
Emotion: sadness (confidence: 0.9975)
All emotions: {'sadness': 0.9974863529205322, 'joy': 0.0003157694300170988, 'love': 0.00019920531485695392, 'anger': 0.0017062496626749635, 'fear': 0.00014795355673413724, 'surprise': 0.0001444307272322476}

Sentence: 'That movie was so terrifying, I couldn't sleep.'
Emotion: fear (confidence: 0.9328)